TUAN DUONG COMPANY

In [ ]:
#import libraries
import pandas as pd
import numpy as np
import pymysql

import math 
#test more


PHASE 1: DATA CLEANING
Information about sales, orders, chats, products and traffic are exported to seperated excel files for each month from December 2022 to February 2023. 
In this phase, I will merge sales data of 3 months into one file and clean the data so that it can be used for calculation in the later phases. The same will be done for orders, chats, products and traffic datasets. Therefore, by the end of this phases, there will be 5 excel files:
- sales overview data from December 2022 to February 2023
- all orders data from December 2022 to February 2023
- chats data from December 2022 to February 2023
- products overview data from December 2022 to February 2023
- traffic overview data from December 2022 to February 2023

In [ ]:
#convert string to float/integer: 106.012,55 (str) -> 106012.55 (float)
def string_to_numeric(df,column):
     df[column] = pd.to_numeric(df[column].str.replace('.', '').str.replace(',', '.'), errors='coerce')

#convert datatype of percentage from string to float: 82,98%(str) -> 0.8298 (float)
def convert_percentage(df,column):
    df[column] = pd.to_numeric(df[column].str.replace(',', '.').str.replace('%', ''), errors='coerce')
    df[column] = df[column]/100

# convert data from float to integer (SKU)
def convert_to_string(df, column):
    df[column] = df[column].fillna(0).astype(int)
    
#find and replace nan, blank values with None
def fill_nan(df):
    df = df.fillna('None',inplace=True)
    
#convert datetime from dd-mm-yyyy to yyyy-mm-dd    
def convert_datetime(df, column):
    df[column] = pd.to_datetime(df[column], errors='coerce', format='%d-%m-%Y')

#convert string 'h-m-s' to time 'h-m-s'
def convert_hms_time(df, column):
    df[column] = pd.to_datetime(df[column],errors='coerce', format='%H:%M:%S').dt.time

In [ ]:
# import and merge
sale_dec = pd.read_excel(r'./Data_December/[export_report]sales_overview_20221201-20221231.xlsx', skiprows = [0,1,2])
sale_jan = pd.read_excel(r'./Data_January/[export_report]sales_overview_20230101-20230131.xlsx', skiprows = [0,1,2])
sale_feb = pd.read_excel(r'./Data_February/[export_report]sales_overview_20230201-20230228.xlsx',skiprows = [0,1,2])
        
sale_dec_jan_feb = pd.concat([sale_dec, sale_jan, sale_feb], ignore_index = True)
    
string_to_numeric(sale_dec_jan_feb, 'Doanh số (Đơn đã đặt) (VND)')
string_to_numeric(sale_dec_jan_feb, 'Doanh số (Đơn đã xác nhận) (VND)')
string_to_numeric(sale_dec_jan_feb, 'Doanh số trên mỗi Người mua (Đơn đã xác nhận) (VND)')
    
convert_percentage(sale_dec_jan_feb, 'Tỷ lệ chuyển đổi')
convert_percentage(sale_dec_jan_feb, 'Tỷ lệ chuyển đổi (Từ Lượt truy cập thành Đơn được đặt)')
convert_percentage(sale_dec_jan_feb, 'Tỷ lệ chuyển đổi (Từ Đơn đã đặt thành Đơn đã xác nhận)')   

convert_datetime(sale_dec_jan_feb, 'Ngày')

fill_nan(sale_dec_jan_feb)

sale_dec_jan_feb.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 90 entries, 0 to 89
Data columns (total 14 columns):
 #   Column                                                  Non-Null Count  Dtype         
---  ------                                                  --------------  -----         
 0   Ngày                                                    90 non-null     datetime64[ns]
 1   Lượt truy cập                                           90 non-null     int64         
 2   Khách đã đặt hàng, chưa thanh toán                      90 non-null     int64         
 3   Sản phẩm khách chưa thanh toán                          90 non-null     int64         
 4   Tất cả các đơn                                          90 non-null     int64         
 5   Doanh số (Đơn đã đặt) (VND)                             90 non-null     int64         
 6   Tỷ lệ chuyển đổi (Từ Lượt truy cập thành Đơn được đặt)  90 non-null     float64       
 7   Người mua có đơn đã xác nhận                            90 non-n

/Users/kayeng/opt/anaconda3/lib/python3.9/site-packages/openpyxl/styles/stylesheet.py:226: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")
/Users/kayeng/opt/anaconda3/lib/python3.9/site-packages/openpyxl/styles/stylesheet.py:226: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")
/Users/kayeng/opt/anaconda3/lib/python3.9/site-packages/openpyxl/styles/stylesheet.py:226: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")
/var/folders/xn/dpmpz1295mbbnwf7r85k280m0000gp/T/ipykernel_20160/2315644471.py:3: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  df[col

In [ ]:
#import and merge
orders_dec = pd.read_excel(r'./Data_December/Order.all.20221201_20221231.xlsx')
orders_jan = pd.read_excel(r'./Data_January/Order.all.20230101_20230131.xlsx')
orders_feb = pd.read_excel(r'./Data_February/Order.all.20230201_20230228.xlsx')

orders_dec_jan_feb = pd.concat([orders_dec, orders_jan, orders_feb], ignore_index = True)

#filter only neccessary columns related to products and sales (excluded delivery information)
filtered_orders_1 = orders_dec_jan_feb.iloc[:, [0,1,2,3,4,14,15]]
filtered_orders_2 = orders_dec_jan_feb.iloc[:, 19:42]
filtered_orders_3 = orders_dec_jan_feb.iloc[:, [44,45,46,47,48,52]]
filtered_orders = pd.concat([filtered_orders_1, filtered_orders_2,filtered_orders_3], axis = 1)


convert_to_string(filtered_orders,'SKU sản phẩm')



filtered_orders['Ngày đặt hàng'] = pd.to_datetime(filtered_orders['Ngày đặt hàng'], errors='coerce')

fill_nan(filtered_orders)

filtered_orders.info()




<class 'pandas.core.frame.DataFrame'>
RangeIndex: 11380 entries, 0 to 11379
Data columns (total 36 columns):
 #   Column                                                    Non-Null Count  Dtype         
---  ------                                                    --------------  -----         
 0   Mã đơn hàng                                               11380 non-null  object        
 1   Mã Kiện Hàng                                              11380 non-null  object        
 2   Ngày đặt hàng                                             11380 non-null  datetime64[ns]
 3   Trạng Thái Đơn Hàng                                       11380 non-null  object        
 4   Lý do hủy                                                 11380 non-null  object        
 5   SKU sản phẩm                                              11380 non-null  int64         
 6   Tên sản phẩm                                              11380 non-null  object        
 7   SKU phân loại hàng                      

In [ ]:
#import and merge
chat_dec = pd.read_excel(r'./Data_December/[export_report]Chat_01122022-31122022.xlsx')
chat_jan = pd.read_excel(r'./Data_January/[export_report]Chat_01012023-31012023.xlsx')
chat_feb = pd.read_excel(r'./Data_February/[export_report]Chat_01022023-28022023.xlsx')

chat_dec_jan_feb = pd.concat([chat_dec, chat_jan, chat_feb], ignore_index = True)

#convert data using the fuction created above
string_to_numeric(chat_dec_jan_feb, 'Doanh số(₫)')

convert_percentage(chat_dec_jan_feb, 'Tỷ lệ truy cập có chat')
convert_percentage(chat_dec_jan_feb, 'Tỷ lệ chuyển đổi (Yêu cầu phản hồi)')
convert_percentage(chat_dec_jan_feb, 'Tỷ lệ phản hồi Chat')
convert_percentage(chat_dec_jan_feb, 'Tỷ lệ chuyển đổi (Phản hồi - Đơn được đặt)')

chat_dec_jan_feb = chat_dec_jan_feb.replace('-', None)

convert_datetime(chat_dec_jan_feb, 'Ngày')

convert_hms_time(chat_dec_jan_feb, 'Thời Gian Phản Hồi')


chat_dec_jan_feb.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 90 entries, 0 to 89
Data columns (total 15 columns):
 #   Column                                      Non-Null Count  Dtype         
---  ------                                      --------------  -----         
 0   Ngày                                        90 non-null     datetime64[ns]
 1   Lượt truy cập                               90 non-null     float64       
 2   Lượt Chat                                   90 non-null     int64         
 3   Lượt truy cập có chat                       90 non-null     int64         
 4   Tỷ lệ truy cập có chat                      90 non-null     float64       
 5   Phản Hồi Chat                               90 non-null     int64         
 6   Chat Chưa Phản Hồi                          90 non-null     int64         
 7   Thời Gian Phản Hồi                          81 non-null     object        
 8   Tỷ lệ chuyển đổi (Yêu cầu phản hồi)         90 non-null     float64       
 9   Tỷ lệ phản h

/var/folders/xn/dpmpz1295mbbnwf7r85k280m0000gp/T/ipykernel_20160/2315644471.py:3: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  df[column] = pd.to_numeric(df[column].str.replace('.', '').str.replace(',', '.'), errors='coerce')


In [ ]:
#import and merge
traffic_dec = pd.read_excel(r'./Data_December/[export_report]traffic_overview_20221201_20221231.xlsx', skiprows = [0,1,2])
traffic_jan = pd.read_excel(r'./Data_January/[export_report]traffic_overview_20230101_20230131.xlsx', skiprows = [0,1,2])
traffic_feb = pd.read_excel(r'./Data_February/[export_report]traffic_overview_20230201_20230228.xlsx',skiprows = [0,1,2])

traffic_dec_jan_feb = pd.concat([traffic_dec, traffic_jan, traffic_feb], ignore_index = True)

#convert database using the fuction created above
string_to_numeric(traffic_dec_jan_feb, 'Số lượt xem trung bình')

convert_percentage(traffic_dec_jan_feb, 'Tỉ lệ thoát trang')

convert_datetime(traffic_dec_jan_feb, 'Ngày')

convert_hms_time(traffic_dec_jan_feb, 'Thời gian xem trung bình')


traffic_dec_jan_feb.info()



<class 'pandas.core.frame.DataFrame'>
RangeIndex: 90 entries, 0 to 89
Data columns (total 9 columns):
 #   Column                      Non-Null Count  Dtype         
---  ------                      --------------  -----         
 0   Ngày                        90 non-null     datetime64[ns]
 1   Lượt xem                    90 non-null     float64       
 2   Số lượt xem trung bình      90 non-null     float64       
 3   Thời gian xem trung bình    90 non-null     object        
 4   Tỉ lệ thoát trang           90 non-null     float64       
 5   Lượt truy cập               90 non-null     float64       
 6   Số khách truy cập mới       90 non-null     float64       
 7   Số khách truy cập hiện tại  90 non-null     float64       
 8   Người theo dõi mới          90 non-null     int64         
dtypes: datetime64[ns](1), float64(6), int64(1), object(1)
memory usage: 6.5+ KB


/Users/kayeng/opt/anaconda3/lib/python3.9/site-packages/openpyxl/styles/stylesheet.py:226: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")
/Users/kayeng/opt/anaconda3/lib/python3.9/site-packages/openpyxl/styles/stylesheet.py:226: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")
/Users/kayeng/opt/anaconda3/lib/python3.9/site-packages/openpyxl/styles/stylesheet.py:226: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")
/var/folders/xn/dpmpz1295mbbnwf7r85k280m0000gp/T/ipykernel_20160/2315644471.py:3: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  df[col

In [ ]:
#import and merge
product_dec = pd.read_excel(r'./Data_December/[export_report]productoverview20221201-20221231.xlsx')
product_jan = pd.read_excel(r'./Data_January/[export_report]productoverview20230101-20230131.xlsx')
product_feb = pd.read_excel(r'./Data_February/[export_report]productoverview20230201-20230228.xlsx')

product_dec_jan_feb = pd.concat([product_dec, product_jan, product_feb], ignore_index = True)

#convert database using the fuction created above
string_to_numeric(product_dec_jan_feb, 'Doanh số (Đơn đã đặt) (VND)')
string_to_numeric(product_dec_jan_feb, 'Doanh số (Đơn đã xác nhận) (VND)')

convert_percentage(product_dec_jan_feb, 'Tỷ lệ thoát Trang sản phẩm')
convert_percentage(product_dec_jan_feb, 'Tỷ lệ chuyển đổi (theo lượt thêm vào giỏ hàng)')
convert_percentage(product_dec_jan_feb, 'Tỷ lệ chuyển đổi (Đơn đã đặt)')
convert_percentage(product_dec_jan_feb, 'Tỷ lệ chuyển đổi (Đơn đã xác nhận)')
convert_percentage(product_dec_jan_feb, 'Tỷ lệ chuyển đổi (Từ Đơn đã đặt thành Đơn đã xác nhận)')

fill_nan(product_dec_jan_feb)

convert_datetime(product_dec_jan_feb, 'Ngày')

product_dec_jan_feb.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 90 entries, 0 to 89
Data columns (total 22 columns):
 #   Column                                                  Non-Null Count  Dtype         
---  ------                                                  --------------  -----         
 0   Ngày                                                    90 non-null     datetime64[ns]
 1   Lượt truy cập sản phẩm                                  90 non-null     int64         
 2   Lượt xem trang sản phẩm                                 90 non-null     int64         
 3   Sản phẩm được truy cập                                  90 non-null     int64         
 4   Số lượng khách thoát trang sản phẩm                     90 non-null     int64         
 5   Tỷ lệ thoát Trang sản phẩm                              90 non-null     float64       
 6   Lượt click từ Trang tìm kiếm                            90 non-null     int64         
 7   Lượt thích                                              90 non-n

/Users/kayeng/opt/anaconda3/lib/python3.9/site-packages/openpyxl/styles/stylesheet.py:226: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")
/Users/kayeng/opt/anaconda3/lib/python3.9/site-packages/openpyxl/styles/stylesheet.py:226: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")
/Users/kayeng/opt/anaconda3/lib/python3.9/site-packages/openpyxl/styles/stylesheet.py:226: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")
/var/folders/xn/dpmpz1295mbbnwf7r85k280m0000gp/T/ipykernel_20160/2315644471.py:3: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  df[col

PHASE 2: Export a Pandas DataFrame from Jupyter Notebook and send it to MySQL
In this phrase, 5 final dataframes created in phase 1 will be send to MySQL server from Jupyter Notebook.

In [ ]:
#connect to mysql
def connection():
    try:
        cnn = pymysql.connect(host='localhost', user='root', port=3306, database = 'database_kaye')
        return cnn
    except Exception as e:
        print('Exception connection: ', e)

c = connection()

#export and send to MYSQL:
cursor = c.cursor()

sales_data = sale_dec_jan_feb.values.tolist()
query1 = "INSERT INTO sales_overview VALUES (%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s)"
cursor.executemany(query1, sales_data)

orders_data = filtered_orders.values.tolist()
query2 = "INSERT INTO orders_overview VALUES (%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s)"
cursor.executemany(query2, orders_data)

chat_data = chat_dec_jan_feb.values.tolist()
query3 = "INSERT INTO chat_overview VALUES (%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s)"
cursor.executemany(query3, chat_data)

traffic_data = traffic_dec_jan_feb.values.tolist()
query4 = "INSERT INTO traffic_overview VALUES (%s,%s,%s,%s,%s,%s,%s,%s,%s)"
cursor.executemany(query4, traffic_data)

product_data = product_dec_jan_feb.values.tolist()
query5 = "INSERT INTO product_overview VALUES (%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s)"
cursor.executemany(query5, product_data)

c.commit()
c.close()

